# Bab 3: Classification

## 1. Pendahuluan

Bab ini berfokus pada sistem klasifikasi, yang merupakan salah satu tugas utama dalam pembelajaran terawasi (*supervised learning*) selain regresi. Dalam bab ini, konsep-konsep dasar klasifikasi dijelaskan secara mendalam, terutama dengan menggunakan dataset MNIST yang populer.

---

## 2. Dataset MNIST

Dataset MNIST diperkenalkan sebagai "hello world" di dunia Machine Learning untuk tugas klasifikasi. Dataset ini terdiri dari 70.000 gambar kecil tulisan tangan digit (0-9), dengan setiap gambar berukuran 28x28 piksel dan memiliki 784 fitur (intensitas piksel dari 0 hingga 255). Dataset ini sudah dibagi menjadi 60.000 gambar untuk training dan 10.000 gambar untuk testing.

Pertama, kita muat dataset MNIST. `keras` menyediakan fungsi praktis untuk ini.

```python
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt

# Memuat dataset MNIST
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

# Mengubah gambar 28x28 menjadi vektor 1D berukuran 784
X_train = X_train_full.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

# Normalisasi data (opsional, tetapi sering membantu)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

y_train = y_train_full

# Tampilkan salah satu digit
some_digit_img = X_train[0].reshape(28, 28)
plt.imshow(some_digit_img, cmap="binary")
plt.axis("off")
plt.show()
```

---

### 3 Melatih Pengklasifikasi Biner (Binary Classifier)

Konsep klasifikasi biner dijelaskan dengan contoh "pendeteksi angka 5", di mana tujuannya adalah membedakan gambar angka 5 dari bukan angka 5. Untuk tugas ini, digunakan pengklasifikasi Stochastic Gradient Descent (SGDClassifier) dari Scikit-Learn. SGDClassifier dipilih karena kemampuannya menangani dataset yang sangat besar secara efisien.

```python
from sklearn.linear_model import SGDClassifier

# Membuat label target untuk klasifikasi biner (True untuk 5, False untuk lainnya)
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

# Membuat dan melatih model SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

# Menguji pada satu instance (digit pertama, yaitu angka 5)
some_digit = X_train[0]
print(f"Prediksi untuk digit pertama (seharusnya True): {sgd_clf.predict([some_digit])}")
```

---

### 4. Pengukuran Kinerja (Perfomance Measures)

Evaluasi kinerja pengklasifikasi adalah bagian penting, mengingat bahwa akurasi saja seringkali bukan metrik yang memadai, terutama untuk dataset yang tidak seimbang (skewed dataset).

**Akurasi Menggunakan Validasi silang (Cross- Validation)**

Akurasi saja bisa menipu. Sebagai contoh, pengklasifikasi "bodoh" yang selalu memprediksi "bukan angka 5" bisa mencapai akurasi 90% pada dataset MNIST, karena hanya sekitar 10% data yang merupakan angka 5.

```python
from sklearn.model_selection import cross_val_score

# Mengukur akurasi SGDClassifier
accuracy_sgd = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
print(f"Akurasi SGD Classifier: {accuracy_sgd}")

# Contoh "pengklasifikasi bodoh"
from sklearn.base import BaseEstimator
class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        return self
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()
accuracy_dumb = cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring="accuracy")
print(f"Akurasi 'Never 5' Classifier: {accuracy_dumb}")
```

**Confusion Matrix**

Confusion Matrix adalah cara yang lebih baik untuk mengevaluasi kinerja. Matriks ini menghitung berapa kali instance dari kelas A diklasifikasikan sebagai kelas B. Istilah kunci:
* True Negative (TN): Instance negatif yang diprediksi dengan benar.
* False Positive (FP): Instance negatif yang salah diprediksi sebagai positif.
* False Negative (FN): Instance positif yang salah diprediksi sebagai negatif.
* True Positive (TP): Instance positif yang diprediksi dengan benar.

```python
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# Mendapatkan prediksi untuk setiap instance di training set
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

# Menghitung confusion matrix
cm = confusion_matrix(y_train_5, y_train_pred)
print("Confusion Matrix:\n", cm)
```

**Presisi (Precision) dan Recall**

Dua metrik penting dari Confusion matrix:
* Precision (Presisi): Akurasi dari prediksi positif (TP / (TP + FP)). Menjawab pertanyaan: "Dari semua yang diprediksi sebagai kelas positif, berapa persen yang benar?"
* Recall (Sensitivitas): Rasio instance positif yang berhasil dideteksi oleh model (TP / (TP + FN)). Menjawab pertanyaan: "Dari semua instance positif yang sebenarnya, berapa persen yang berhasil terdeteksi?"
* F1 Score: Rata-rata harmonik dari precision dan recall, berguna untuk membandingkan model yang memiliki presisi dan recall seimbang.

```python
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision:", precision_score(y_train_5, y_train_pred))
print("Recall:", recall_score(y_train_5, y_train_pred))
print("F1 Score:", f1_score(y_train_5, y_train_pred))
```

**Trade-off Presisi/Recall**

Tidak mungkin memiliki presisi dan recall yang tinggi secara bersamaan; meningkatkan satu akan mengurangi yang lain. Hal ini dipengaruhi oleh decision threshold yang digunakan model. Dengan mengubah threshold, kita bisa mengatur trade-off ini.

```python
from sklearn.metrics import precision_recall_curve

# Mendapatkan decision scores alih-alih prediksi
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

# Menghitung presisi dan recall untuk berbagai threshold
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.xlabel("Threshold")
    plt.legend(loc="center left")
    plt.ylim([0, 1])
    plt.grid(True)
    plt.title("Precision and Recall vs. Decision Threshold")

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()
```

**Kurva ROC (Receiver Operating Characteristic)**

Kurva ROC adalah metrik evaluasi populer lainnya. Kurva ini memplot True Positive Rate (Recall) melawan False Positive Rate (FPR). Metrik Area Under the Curve (AUC) sering digunakan untuk membandingkan model. AUC = 1 untuk model sempurna, dan AUC = 0.5 untuk model acak.

```python
from sklearn.metrics import roc_curve, roc_auc_score

# Menghitung FPR, TPR untuk berbagai threshold
fpr, tpr, thresholds_roc = roc_curve(y_train_5, y_scores)

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # Garis putus-putus untuk pengklasifikasi acak
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.title("ROC Curve")
plt.show()

# Menghitung Area Under the Curve (AUC)
print("ROC AUC Score:", roc_auc_score(y_train_5, y_scores))
```

---

### 5. Klasifikasi Multikelas (Multiclass Classification)

Sekarang kita akan melatih model untuk mengklasifikasikan semua 10 digit. Beberapa algoritma (seperti SGDClassifier, SVC) secara otomatis menggunakan strategi One-versus-Rest (OvR) atau One-versus-One (OvO) ketika dihadapkan pada tugas multikelas.

```python
# Melatih pada semua 10 kelas (menggunakan y_train, bukan y_train_5)
# SGDClassifier secara otomatis menjalankan OvR
sgd_clf.fit(X_train, y_train) 
print("Prediksi untuk digit pertama (seharusnya 5):", sgd_clf.predict([some_digit]))
```

---

### 6. Analisis Kesalahan (Error Analysis)

Menganalisis jenis kesalahan yang dibuat model dapat membantu kita memperbaikinya. Kita bisa memvisualisasikan confusion matrix untuk melihat pola kesalahan.

```python
from sklearn.preprocessing import StandardScaler

# Scaling dapat meningkatkan performa
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Mendapatkan prediksi multikelas
y_train_pred_multi = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3)

# Membuat confusion matrix multikelas
conf_mx = confusion_matrix(y_train, y_train_pred_multi)
print("Confusion Matrix Multikelas:\n", conf_mx)

# Visualisasi confusion matrix
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.title("Multiclass Confusion Matrix")
plt.show()

# Fokus pada error: normalisasi confusion matrix dan isi diagonal dengan nol
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.title("Normalized Error Matrix")
plt.show()
```

Visualisasi matriks error di atas menunjukkan pasangan angka mana yang sering salah diklasifikasikan (area yang lebih terang).

---

### 7. Klasifikikasi Multilabel (Multilabel Classification)

Ini adalah konsep di mana pengklasifikasi dapat menghasilkan beberapa kelas untuk satu instance (misalnya, mengenali beberapa orang dalam satu gambar).

```python
from sklearn.neighbors import KNeighborsClassifier

# Contoh: label ganda untuk "angka besar" (>=7) dan "angka ganjil"
y_train_large = (y_train >= 7)
y_train_odd = (y_train % 2 == 1)
y_multilabel = np.c_[y_train_large, y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

print("Prediksi multilabel untuk digit pertama (5):", knn_clf.predict([some_digit]))
# Output: [[False, True]] -> Bukan angka besar, tapi angka ganjil
```

---

### 8. Klasifikasi Multioutput (Multioutput Classification)

Ini adalah generalisasi dari klasifikasi multilabel di mana setiap label bisa bersifat multikelas (memiliki lebih dari dua nilai yang mungkin). Contohnya adalah sistem penghilang noise dari gambar, di mana setiap piksel adalah label.

```python
# Menambahkan noise ke gambar MNIST
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise / 255.0
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise / 255.0
y_train_mod = X_train
y_test_mod = X_test

# Melatih classifier untuk membersihkan noise
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[0]])

# Fungsi untuk plot
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary, interpolation="nearest")
    plt.axis("off")

# Tampilkan gambar asli, gambar bernoise, dan gambar yang sudah dibersihkan
plt.figure(figsize=(8,4))
plt.subplot(131); plot_digit(X_test_mod[0]); plt.title("Noisy")
plt.subplot(132); plot_digit(clean_digit); plt.title("Cleaned")
plt.subplot(133); plot_digit(y_test_mod[0]); plt.title("Original")
plt.show()
```